In [83]:
import pandas as pd

def json_parser(data):
  import json
  try:
    j1 = json.loads(data)
    return j1
  except:
    x = '{}'
    return json.loads(x)

df = pd.read_csv('train.csv', converters={'amenities':json_parser}, header=0)
df[sorted(df['amenities'][0].keys())] = df['amenities'].apply(pd.Series)

y = df[['price']]
y.columns


Index(['price'], dtype='object')

In [84]:
df.columns

Index(['id', 'name', 'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_verifications', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'country_code', 'country', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'price', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_ch

In [86]:
df.to_csv('train_w_amenities.csv')

AttributeError: module 'pandas' has no attribute 'to_csv'

In [75]:
import json
from math import isnan

def to_float(s):
    try:
        ret = float(s)
    except:
        ret = -1
    if isnan(ret):
        ret = -1
    return ret

def to_float_avg(s, name_of_col):
    avg = df[name_of_col].mean()
    try:
        ret = float(s)
    except:
        ret = avg
    if isnan(ret):
        ret = avg
    return ret

def to_int(s):
    try:
        ret = int(s)
    except:
        ret = -1
    return ret

def to_int_avg(s, name_of_col):
    avg = df[name_of_col].mean()
    try:
        ret = int(s)
    except:
        ret = avg
    return ret

def json_parser(data):
    j1 = json.loads(data)
    return j1

categories = [x for x in list(set(df['neighbourhood_group_cleansed'])) if type(x) is str]
state_categories = [x for x in list(set(df['state'])) if type(x) is str]
state_categories = [x for x in list(set(df['state'])) if type(x) is str]
room_categories = [x for x in list(set(df['room_type'])) if type(x) is str]
super_host_categories = [x for x in list(set(df['host_is_superhost'])) if type(x) is str]
bed_type_categories = [x for x in list(set(df['bed_type'])) if type(x) is str]
cancellation_policy_categories = [x for x in list(set(df['cancellation_policy'])) if type(x) is str]
property_type_categories = [x for x in list(set(df['property_type'])) if type(x) is str]
print(len(categories))

def create_feature(row):
    guest_num = to_int(row.guests_included)
    guest_cap = to_float(row.extra_people)
    bedrooms = to_int_avg(row.bedrooms, 'bedrooms')
    bathrooms = to_int_avg(row.bathrooms, 'bathrooms')
    beds = to_int_avg(row.beds, 'beds')
    review = to_float(row.review_scores_rating)
    review_scores_location = to_float(row.review_scores_location)
    cleanliness_review = to_int(row.review_scores_cleanliness)
    number_of_reviews = to_int_avg(row.number_of_reviews, 'number_of_reviews')
    host_listings = to_int(row.calculated_host_listings_count)
    accommodates = to_int_avg(row.accommodates, 'accommodates')
    reviews_per_month = to_float_avg(row.reviews_per_month, 'reviews_per_month')
    one_hot = [int(row.neighbourhood_group_cleansed == category) for category in categories]
    two_hot = [int(row.state == state_category) for state_category in state_categories]
    three_hot = [int(row.room_type == room_category) for room_category in room_categories]
    four_hot = [int(row.host_is_superhost == super_host_category) for super_host_category in super_host_categories]
    five_hot = [int(row.bed_type == bed_type_category) for bed_type_category in bed_type_categories]
    six_hot = [int(row.cancellation_policy == cancelation_policy_category) for cancelation_policy_category in cancellation_policy_categories]
    seven_hot = [int(row.property_type == property_type_category) for property_type_category in property_type_categories]
    
    return [guest_num, guest_cap, accommodates, bedrooms, bathrooms, beds, number_of_reviews, reviews_per_month] + one_hot + three_hot + four_hot + five_hot + seven_hot

train_X, train_y = [], []
for (idx, row) in df.iterrows():
    price = to_float(row.price)
    feature = create_feature(row)

    train_X.append(feature)
    train_y.append(price)

print(len(train_X), len(train_y))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [7]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy as np



In [ ]:

# This is for finding best alpha value
# 1.5 seems to a be a good value
import matplotlib.pyplot as plt
scaler_X = StandardScaler().fit(train_X)
scaler_X.transform(train_X)

scores = []
alphas = []
for i in np.linspace(1.5, 4.0, 20):
  print(i)
  score = cross_val_score(MLPRegressor(activation='logistic', alpha=i, hidden_layer_sizes=(8,), random_state=1, max_iter=1600), train_X, train_y, cv=3, n_jobs=8)
  scores.append(np.mean(score))
  alphas.append(i) 
plt.plot(alphas, scores)
plt.show()

In [71]:


# This cell is used for cross validation of the model

# scaler_X = StandardScaler().fit(train_X)
# scaler_X.transform(train_X)

# With standardized X
# Setting up the neural net classifier
regr = MLPRegressor(activation='logistic', alpha=1.4, hidden_layer_sizes=(100,80,30), random_state=1, max_iter=1800)

print(np.mean(cross_val_score(regr, train_X, train_y, cv=5, n_jobs=12)))

bestSize = 0
bestAvg = 0

#for i in range(1, 50):
#  score = cross_val_score(MLPRegressor(activation='relu', alpha=1e-05, hidden_layer_sizes=(i,), random_state=1, max_iter=300), train_X, train_y, cv=5, n_jobs=4)
#  if(np.mean(score) > bestAvg):
#    bestAvg = np.mean(score)
#    bestSize = i


0.4784783345173468


In [53]:
regr.fit(train_X, train_y)

MLPRegressor(activation='logistic', alpha=1.4, hidden_layer_sizes=(200, 50, 20),
             max_iter=1800, random_state=1)

In [44]:
test_df = pd.read_csv('test.csv')
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
  feature = create_feature(row)
  test_ids.append(row.id)
  test_X.append(feature)
test_y = regr.predict(test_X)


output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('mlpregressor_prediction.csv', index=False)



In [57]:

scaler_X = StandardScaler().fit(train_X)
scaler_X.transform(train_X)

# With standardized X
# Setting up the neural net classifier
regr = MLPRegressor(activation='logistic', alpha=1.4, hidden_layer_sizes=(120,100,30), random_state=1, max_iter=1800)

print(np.mean(cross_val_score(regr, train_X, train_y, cv=5, n_jobs=12)))

bestSize = 0
bestAvg = 0

#for i in range(1, 50):
#  score = cross_val_score(MLPRegressor(activation='relu', alpha=1e-05, hidden_layer_sizes=(i,), random_state=1, max_iter=300), train_X, train_y, cv=5, n_jobs=4)
#  if(np.mean(score) > bestAvg):
#    bestAvg = np.mean(score)
#    bestSize = i


0.5015955766426005


In [20]:


# This cell is used for cross validation of the model
scaler_X = StandardScaler().fit(train_X)
scaler_X.transform(train_X)

# With standardized X
# Setting up the neural net classifier
regr = MLPRegressor(activation='logistic', alpha=1.4, hidden_layer_sizes=(180,120,30), random_state=1, max_iter=1800)

print(np.mean(cross_val_score(regr, train_X, train_y, cv=5, n_jobs=12)))

bestSize = 0
bestAvg = 0

#for i in range(1, 50):
#  score = cross_val_score(MLPRegressor(activation='relu', alpha=1e-05, hidden_layer_sizes=(i,), random_state=1, max_iter=300), train_X, train_y, cv=5, n_jobs=4)
#  if(np.mean(score) > bestAvg):
#    bestAvg = np.mean(score)
#    bestSize = i


0.5515102997703232
